# MNIST

### MNIST 데이터 받기

In [1]:
import os
import sys
from six.moves import urllib
import gzip
import pickle
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt


SOURCE_URL = 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
FILENAME = SOURCE_URL.split('/')[-1]
DATA_DIR = './datasets'

def maybe_download(data_dir):
    filepath = os.path.join(data_dir, FILENAME)
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    if not os.path.isfile(filepath):
        def _progress(count, block_size, total_size):
            sys.stdout.write('\r>> Downloading {} {:.1f} %'.format(
                FILENAME, float(count * block_size) / float(total_size) * 100.0))
            sys.stdout.flush()
        filepath, _ = urllib.request.urlretrieve(SOURCE_URL, filepath, _progress)
        print()
        statinfo = os.stat(filepath)
        print('Successfully donloaded', FILENAME, statinfo.st_size, 'bytes.')

def load(data_dir, subset='train'):
    maybe_download(data_dir)
    filepath = os.path.join(data_dir, FILENAME)
    
    f = gzip.open(filepath, 'rb')
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    train_set, valid_set, test_set = u.load()
    f.close()
    
    if subset == 'train':
        trainx, trainy = train_set
        trainx = trainx.astype(np.float32).reshape(trainx.shape[0], 28, 28)
        trainy = trainy.astype(np.uint8)
        return trainx, trainy
    elif subset == 'test':
        testx, testy = test_set
        testx = testx.astype(np.float32).reshape(testx.shape[0], 28, 28)
        testy = testy.astype(np.uint8)
        return testx, testy
    elif subset== 'valid':
        validx, validy = valid_set
        validx = validx.astype(np.float32).reshape(validx.shape[0], 28, 28)
        validy = validy.astype(np.uint8)
        return validx, validy
    else:
        raise NotImplementedError('subset should be train or valid or test')

# Load data
train_data, train_label = load(DATA_DIR, 'train')
valid_data, valid_label = load(DATA_DIR, 'valid')
test_data, test_label = load(DATA_DIR, 'test')

# concatenate train and valid data as train data
train_data = np.concatenate((train_data, valid_data))
train_label = np.concatenate((train_label, valid_label))

### MNIST 데이터 확인 하기

In [2]:
# size of MNIST
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
# # show data
# _, (ax1, ax2) = plt.subplots(1, 2)
# sample_data = train_data[728]
# ax1.imshow(sample_data, cmap=plt.cm.Greys);
# ax2.hist(sample_data, bins=20, range=[0, 1]);

In [4]:
import sklearn
from sklearn.linear_model import LogisticRegression

In [5]:
# model_log = LogisticRegression()
# model_log.fit(np.reshape(train_data, (60000)))

In [6]:
import tensorflow as tf

In [7]:
def model(X, by):
    X = tf.expand_dims(X, axis = 3)
    with tf.variable_scope('first'):
        outs = tf.layers.conv2d(X, 128, 3, padding='same')
        outs = tf.nn.relu(outs)
        outs = tf.layers.max_pooling2d(outs, 2, 2)
    with tf.variable_scope('second'):
        outs = tf.layers.conv2d(outs, 256, 3,padding='same')
        outs = tf.nn.relu(outs)
        outs = tf.layers.max_pooling2d(outs, 2, 2)
        
    with tf.variable_scope('third'):
        outs = tf.layers.conv2d(outs, 128, 3,padding='same')
        outs = tf.nn.relu(outs)
        outs = tf.layers.max_pooling2d(outs, 2, 2)
    print(outs)
    
#flattening before fc
    outs = tf.reshape(outs,
                      (-1,
                       outs.shape[1] * outs.shape[2] * outs.shape[3]))
    outs = tf.layers.dense(outs, 256)
    outs = tf.nn.relu(outs)
    outs = tf.layers.dense(outs, 10)

#soft max optimazation
    one_hot = tf.one_hot(by, 10)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot, logits=outs)
    loss = tf.reduce_mean(loss)
    opt = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
    preds = tf.cast(tf.argmax(tf.nn.softmax(outs), axis = 1), tf.int32) # 가장 큰 값의 index를 return axis 방향으로
#     acc = tf.metrics.accuracy(by, preds)
    acc = tf.reduce_mean(tf.cast(tf.equal(by, preds), tf.float32))
    return {
        'loss' : loss,
        'opt' : opt,
        'preds' : preds,
        'acc' : acc
    }

In [8]:
X = tf.placeholder(tf.float32, shape=(None, 28, 28))
by = tf.placeholder(tf.int32)
ours = model(X, by)

init = tf.global_variables_initializer()

Tensor("third/max_pooling2d/MaxPool:0", shape=(?, 3, 3, 128), dtype=float32)


In [9]:
num_epochs = 2
batch_size = 100 #2진수로 안해도 됌 원래는 batch 뽑고 shuffle를 반복해야됨!!
num_display = 100
num_samples = 60000

In [16]:
def evaluate(X_sample, y_sample, batch_size=512):
    """Run a minibatch accuracy op"""
    
    N = X_sample.shape[0]
    correct_sample = 0

    for i in range(0, N, batch_size):
        X_batch = X_sample[i: i + batch_size]
        y_batch = y_sample[i: i + batch_size]
        N_batch = X_batch.shape[0]

        feed = {
            X: X_batch,
            by: y_batch
        }
        accuracy_ = ours['acc']
        correct_sample += sess.run(accuracy_ , feed_dict=feed) * N_batch

    return correct_sample / N

In [14]:
sess = tf.Session()
sess.run(init)
for ind_epoch in range(0, num_epochs):
    print('Current iteration {}'.format(ind_epoch+1))
    for ind_ in range(0, int(num_samples / batch_size)):
        batch_X = train_data[ind_*batch_size:(ind_+1)*batch_size]
        batch_by = train_label[ind_*batch_size:(ind_+1)*batch_size]

        _, cur_loss, cur_acc = sess.run([ours['opt'], ours['loss'], ours['acc']],
                                        feed_dict={X:batch_X, by:batch_by})
        if ind_ % num_display == 0 :
            print('loss :{0:.4f}, acc : {1:.4f}'.format(cur_loss,
                                                            cur_acc))
    #test
#     _, cur_loss, cur_acc = sess.run([ours['opt'], ours['loss'], ours['acc']],
#                                             feed_dict={X:test_data, by:test_label})
#     print('Test3 loss :{0:.4f}, acc : {1:.4f}'.format(cur_loss,
#                                                             cur_acc))
                

Current iteration 1
loss :2.2969, acc : 0.1000
loss :0.1877, acc : 0.9400
loss :0.1014, acc : 0.9700
loss :0.0856, acc : 0.9900
loss :0.0242, acc : 0.9900
loss :0.0147, acc : 1.0000
Current iteration 2
loss :0.0392, acc : 0.9900
loss :0.0305, acc : 0.9900
loss :0.0555, acc : 0.9900
loss :0.0613, acc : 0.9900
loss :0.0143, acc : 0.9900
loss :0.0074, acc : 1.0000


In [21]:
# sess = tf.Session()
evaluate(test_data, test_label,batch_size=100)


0.9866000080108642

In [ ]:
tf.test.is_gpu_available()

In [ ]:
tf.test.gpu_device_name()